# 概要
* 結局基本のOpenAIの素のAPIを触る

# 参考
* https://platform.openai.com/docs/api-reference/completions

In [1]:
import os
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

In [2]:
#Jupyterでも実行できるのは知らんかった...
import logging
import sys
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG, force=True)

# complementation
* レスポンスはJSON形式
* レスポンスの日本語が途中で切れる場合は、total_tokensを確認する
* total_tokensがmax_tokensを下回っている場合は途中で回答が切れる
* total_tokensにはprompt_tokensとcompletion_tokensの合算値からなる
* max_tokensの上限は4096
* TransformersはN^2でメモリを消費するため長い文章の取り扱いが苦手
* max_tokensには4096 - promt_tokensを指定する必要があることに注意
* つまり最大値を指定するなら、max_tokensを下げて指定する必要がある
* promptが文字列でAIPの外側で定義して、それをAPIに渡して実行させる
* prromptは最初と最後を"""で囲むと改行を意識せずに済むので楽

In [15]:
prompt = "初音ミクは"

res = openai.Completion.create(
    model="text-davinci-003",
    prompt=prompt,
    max_tokens=200,
    temperature=0
)

print(res)
print(res.choices[0].text)

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"model": "text-davinci-003", "prompt": "\\u521d\\u97f3\\u30df\\u30af\\u306f", "max_tokens": 200, "temperature": 0}' message='Post details'
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=5482 request_id=ed143ae186bb91949fce9266737282ef response_code=200
{
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": "\u3001\u65e5\u672c\u306e\u30f4\u30a9\u30fc\u30ab\u30ed\u30a4\u30c9\u3067\u3042\u308b\u3002\u5f7c\u5973\u306f\u3001Crypton Future Media\u304c\u958b\u767a\u3057\u305f\u521d\u97f3\u30df\u30af\u30fb\u30d7\u30ed\u30b8\u30a7\u30af\u30c8\u306b\u3088\u3063\u3066\u30012007\u5e748\u670831\u65e5\u306b\u767a\u8868\u3055\u308c\u305f\u3002\u5f7c\u5973\u306f\u

## 長文回答させると壊れることがある

In [21]:
prompt = "初音ミクに関して3000文字程度で教えて下さい"

res = openai.Completion.create(
    model="text-davinci-003",
    prompt=prompt,
    max_tokens=4000,
    temperature=0
)

print(res)
print(res.choices[0].text)

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"model": "text-davinci-003", "prompt": "\\u521d\\u97f3\\u30df\\u30af\\u306b\\u95a2\\u3057\\u30663000\\u6587\\u5b57\\u7a0b\\u5ea6\\u3067\\u6559\\u3048\\u3066\\u4e0b\\u3055\\u3044", "max_tokens": 4000, "temperature": 0}' message='Post details'
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=119223 request_id=05156788c5892eaf18d43c0500f8f4d3 response_code=200
{
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": "\n\n\u521d\u97f3\u30df\u30af\u306f\u30012007\u5e74\u306bCrypton Future Media\u304b\u3089\u767a\u58f2\u3055\u308c\u305f\u97f3\u58f0\u5408\u6210\u30bd\u30d5\u30c8\u30a6\u30a7\u30a2\u300cVOCALOID2\u300d\u306b\u767b\u5834\u3057\u305f3D\u30ad\u30e

## プロンプトエンジニアリング

In [24]:
prompt = "初音ミクに関して3点で教えて下さい\
ポイント1:\
ポイント2:\
ポイント3:\
"

res = openai.Completion.create(
    model="text-davinci-003",
    prompt=prompt,
    max_tokens=500,
    temperature=0
)

print(res)
print(res.choices[0].text)

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"model": "text-davinci-003", "prompt": "\\u521d\\u97f3\\u30df\\u30af\\u306b\\u95a2\\u3057\\u30663\\u70b9\\u3067\\u6559\\u3048\\u3066\\u4e0b\\u3055\\u3044\\u30dd\\u30a4\\u30f3\\u30c81:\\u30dd\\u30a4\\u30f3\\u30c82:\\u30dd\\u30a4\\u30f3\\u30c83:", "max_tokens": 500, "temperature": 0}' message='Post details'
DEBUG:urllib3.connectionpool:Resetting dropped connection: api.openai.com
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=6668 request_id=09fbf0574f78c85ffb91bd81a12430ec response_code=200
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\n\n\u30dd\u30a4\u30f3\u30c81:\u521d\u97f3\u30df\u30af\u306f\u3001\u65e5\u672c\u306e\u30af\u30ea\u30d7\u30c

In [26]:
prompt = "日本語を英語に訳してください\
日本語:吾輩は猫である\
英語:\
"

res = openai.Completion.create(
    model="text-davinci-003",
    prompt=prompt,
    max_tokens=500,
    temperature=0
)

print(res)
print(res.choices[0].text)

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"model": "text-davinci-003", "prompt": "\\u65e5\\u672c\\u8a9e\\u3092\\u82f1\\u8a9e\\u306b\\u8a33\\u3057\\u3066\\u304f\\u3060\\u3055\\u3044\\u65e5\\u672c\\u8a9e:\\u543e\\u8f29\\u306f\\u732b\\u3067\\u3042\\u308b\\u82f1\\u8a9e:", "max_tokens": 500, "temperature": 0}' message='Post details'
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 272
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=420 request_id=fe73662f594d474162d99f7c81d308b8 response_code=200
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "I am a cat."
    }
  ],
  "created": 1678373401,
  "id": "cmpl-6sBsfa0JH5swhx9kcbysk4KJGgSCq",
  "model": "text-davinci-003",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 5,
    

## few shot

In [27]:
prompt = "Convert movie titles into emoji.\
Back to the Future: 👨👴🚗🕒 \
Batman: 🤵🦇 \
Transformers: 🚗🤖 \
Star Wars:\
"

res = openai.Completion.create(
    model="text-davinci-003",
    prompt=prompt,
    max_tokens=500,
    temperature=0
)

print(res)
print(res.choices[0].text)

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"model": "text-davinci-003", "prompt": "Convert movie titles into emoji.Back to the Future: \\ud83d\\udc68\\ud83d\\udc74\\ud83d\\ude97\\ud83d\\udd52 Batman: \\ud83e\\udd35\\ud83e\\udd87 Transformers: \\ud83d\\ude97\\ud83e\\udd16 Star Wars:", "max_tokens": 500, "temperature": 0}' message='Post details'
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=348 request_id=547ed2858429936b10cc600768c37939 response_code=200
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": " \u2b50\ufe0f\ud83c\udf0c"
    }
  ],
  "created": 1678373475,
  "id": "cmpl-6sBtrOr5rdLq0UlLmoLTKaAU0mnaJ",
  "model": "text-davinci-003",
  "object": "text_completion",
  "usage": {
  

In [28]:
prompt = "以下を実施してください\
日本語:吾輩は猫である\
英語:I am a cat.\
日本語:吾輩は犬である\
英語:\
"

res = openai.Completion.create(
    model="text-davinci-003",
    prompt=prompt,
    max_tokens=500,
    temperature=0
)

print(res)
print(res.choices[0].text)

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"model": "text-davinci-003", "prompt": "\\u4ee5\\u4e0b\\u3092\\u5b9f\\u65bd\\u3057\\u3066\\u304f\\u3060\\u3055\\u3044\\u65e5\\u672c\\u8a9e:\\u543e\\u8f29\\u306f\\u732b\\u3067\\u3042\\u308b\\u82f1\\u8a9e:I am a cat.\\u65e5\\u672c\\u8a9e:\\u543e\\u8f29\\u306f\\u72ac\\u3067\\u3042\\u308b\\u82f1\\u8a9e:", "max_tokens": 500, "temperature": 0}' message='Post details'
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 272
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=322 request_id=b8133c7a538d689127a51c50c160b48c response_code=200
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "I am a dog."
    }
  ],
  "created": 1678373549,
  "id": "cmpl-6sBv36iF6FigkbDpqLxxkcO5HOb8w",
  "model": "text-davinci-003",
